In [1]:

####################################################################################################
####################################################################################################

import sys
import os

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint

from IPython.display import clear_output
from IPython.display import display

sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../pax"))

from pax_utils import event_utils
from pax_utils import file_utils
from pax_utils import interaction_utils
from pax_utils import s1s2_utils
from pax_utils import waveform_utils

from pax import core

pp = pprint.PrettyPrinter(depth=6)



In [2]:

####################################################################################################
####################################################################################################

def processPklEvents(zipfilename):

    ################################################################################################
    ################################################################################################
    
    lst_pkl_files  = file_utils.getListOfPklFilesFromZipFile(zipfilename, nEventsPerFile)

    
    ################################################################################################
    ################################################################################################
    
    event         = None

    df_zip_merged = pd.DataFrame()
    #df_zip_event  = pd.DataFrame()
    #df_zip_intr   = pd.DataFrame()
    #df_zip_s2s    = pd.DataFrame()
    
        
    ################################################################################################
    ################################################################################################
    
    for iPklFile in range(0, len(lst_pkl_files)):
    
        print("File: '" + str(zipfilename) + "'")
        print("   event_number: " + str(iPklFile))
        clear_output(wait=True)
        
        
        ############################################################################################
        ############################################################################################
        
        pklfilename  = lst_pkl_files[iPklFile]
        event        = file_utils.getPaxEventFromPklFileInZipArchive(zipfilename, pklfilename)

        
        ############################################################################################
        ############################################################################################
        
        interactions  = event.interactions
        nInteractions = len(interactions)
        
            
        ############################################################################################
        ############################################################################################

        sum_wf = []
        
        #if (nInteractions != 1): continue
        #   
        #if (len(interactions) > 0):
        #    
        #    intr = interactions[0]
        #    
        #    sum_wf = event.peaks[intr.s2].sum_waveform_top
        #        
            
        ############################################################################################
        ############################################################################################
        
        df_pkl_event  = event_utils.getEventDataFrameFromEvent(event)
        df_pkl_intr   = interaction_utils.getInteractionDataFrameFromEvent(event)
        df_pkl_s2s    = s1s2_utils.getS2integralsDataFrame(event, 127)
        
        #display(df_pkl_event)
        #display(df_pkl_intr)
        #display(df_pkl_s2s)
        
     
        ############################################################################################
        ############################################################################################
    
        #df_zip_event  = df_zip_event.append (df_pkl_event)
        #df_zip_intr   = df_zip_intr.append  (df_pkl_intr)
        #df_zip_s2s    = df_zip_s2s.append   (df_pkl_s2s)
        
        df_pkl_merged = df_pkl_event.merge(df_pkl_intr).merge(df_pkl_s2s)
        df_zip_merged = df_zip_merged.append(df_pkl_merged)
        
        continue
      
    
    
        ############################################################################################
        # Check Summed Waveforms
        ############################################################################################

        df_test    = waveform_utils.getS2waveformsAllChannels(event)
        arr_sum_wf = waveform_utils.getSummedWaveform(df_test)
        sumwf      = np.asarray(event.sum_waveforms)

        if (False):
            
            #print("Sum WF Length:      " + str(len(sum_wf)))
            #print("Sum WF Length Test: " + str(arr_sum_wf.shape))
            
            plt.plot(sum_wf)
            plt.plot(arr_sum_wf)
            plt.plot(event.main_s2.sum_waveform_top)

        
        ############################################################################################
        ############################################################################################
          
        #print()
        #display(df_pkl_s2s)
        
        
        #arr_s2_integrals = df_pkl_s2s.drop('event_number').as_matrix()
        
        #print("len(arr_s2_integrals): " + str(len(arr_s2_integrals)))
        #print("arr_s2_integrals:      " + str(arr_s2_integrals))
        #print("S2 Integrals Test: " + str(s2_integral))

        
        ############################################################################################
        ############################################################################################
    
        continue
            
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged.reset_index(inplace=True, drop=True)
       
    return df_zip_merged, event

    

In [60]:

####################################################################################################
####################################################################################################

zipfilename    = '../pax_run/pax_output/instructions_000000/XENON1T-0-000000000-000000999-000001000.zip'
nEventsPerFile = 1000

df_zip_merged, event = processPklEvents(zipfilename)

#wfs = event.sum_waveforms

display(df_zip_merged)

#print(len(event.peaks))
#print(len(event.pulses))
#print(len(wfs))





KeyboardInterrupt: 

In [3]:
import glob

####################################################################################################
####################################################################################################

#ver = 'test'
ver = datetime.datetime.now().strftime("%y%m%d%H%M")

nEventsPerFile = 1000
nFilesZip      = 30

dir_input    = '../pax_run/pax_output/'
dir_format   = dir_input + "instructions_" + ('[0-9]' * 6)
file_format  = 'XENON1T-0-000000000-000000999-000001000.zip'
file_pkl     = 'merged/merged_pax_' + ver + '.pkl'
lst_contents = glob.glob(dir_format)

if (nFilesZip == -1):
    
    nFilesZip    = len(lst_contents)

print()
print("PAX input files: " + str(nFilesZip))
print()


####################################################################################################
####################################################################################################

df = pd.DataFrame()

for iZip in range(0, nFilesZip):
    
    ################################################################################################
    ################################################################################################
    
    zipfilename = lst_contents[iZip] + '/' + file_format
    
    if (not os.path.exists(zipfilename)):
        
        print("Error! File: '" + str(zipfilename) + "' does not exist.")
    
        continue
        
    print("File: '" + str(zipfilename) + "'")
    
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged, event = processPklEvents(zipfilename)
    
    df = df.append(df_zip_merged)
    
    continue
    

print("Done")


####################################################################################################
# Write
####################################################################################################

print(file_pkl)

df.to_pickle(file_pkl)



Done
merged/merged_pax_1803031449.pkl
